<a href="https://colab.research.google.com/github/Omrasal7/research_paper_recommendation_system/blob/main/research_paper_recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df=pd.read_csv("/content/drive/MyDrive/arxiv_data.csv")

In [ ]:
df.head()

,titles,summaries,terms
0,Survey on Semantic Stereo Matching / Semantic ...,Stereo matching is one of the widely used tech...,"['cs.CV', 'cs.LG']"
1,FUTURE-AI: Guiding Principles and Consensus Re...,The recent advancements in artificial intellig...,"['cs.CV', 'cs.AI', 'cs.LG']"
2,Enforcing Mutual Consistency of Hard Regions f...,"In this paper, we proposed a novel mutual cons...","['cs.CV', 'cs.AI']"
3,Parameter Decoupling Strategy for Semi-supervi...,Consistency training has proven to be an advan...,['cs.CV']
4,Background-Foreground Segmentation for Interio...,"To ensure safety in automated driving, the cor...","['cs.CV', 'cs.LG']"


In [ ]:
df.tail()

,titles,summaries,terms
51769,Hierarchically-coupled hidden Markov models fo...,We address the problem of analyzing sets of no...,"['stat.ML', 'physics.bio-ph', 'q-bio.QM']"
51770,Blinking Molecule Tracking,We discuss a method for tracking individual mo...,"['cs.CV', 'cs.DM']"
51771,Towards a Mathematical Foundation of Immunolog...,We attempt to set a mathematical foundation of...,"['stat.ML', 'cs.LG', 'q-bio.GN']"
51772,A Semi-Automatic Graph-Based Approach for Dete...,Diffusion Tensor Imaging (DTI) allows estimati...,['cs.CV']
51773,SparseCodePicking: feature extraction in mass ...,Mass spectrometry (MS) is an important techniq...,"['stat.ML', 'physics.med-ph', 'stat.AP', 'stat..."


# Data cleaning and preprocessing

In [ ]:
df.shape

(51774, 3)

In [ ]:
df.isnull().sum()

titles       0
summaries    0
terms        0
dtype: int64

In [ ]:
df.duplicated().sum()

12783

In [ ]:
labels_column=df['terms'].apply(literal_eval)
labels=labels_column.explode().unique()

In [ ]:
print("labels :",labels)
print("length :",len(labels))

labels : ['cs.CV' 'cs.LG' 'cs.AI' ... 'I.2.6; I.5.1; G.3'
 '92E10, 46M20, 94A08, 68U10, 44A12, 55R35' '92E10']
length : 1099


# Recommendation system

In [ ]:
df.drop(columns=['terms','summaries'],inplace=True)

In [ ]:
df.drop_duplicates(inplace=True)
df.reset_index(drop=True,inplace=True)

In [ ]:
pd.set_option('display.max_colwidth',None)
df

,titles
0,Survey on Semantic Stereo Matching / Semantic Depth Estimation
1,FUTURE-AI: Guiding Principles and Consensus Recommendations for Trustworthy Artificial Intelligence in Future Medical Imaging
2,Enforcing Mutual Consistency of Hard Regions for Semi-supervised Medical Image Segmentation
3,Parameter Decoupling Strategy for Semi-supervised 3D Left Atrium Segmentation
4,Background-Foreground Segmentation for Interior Sensing in Automotive Industry
...,...
38967,A Ray-based Approach for Boundary Estimation of Fiber Bundles Derived from Diffusion Tensor Imaging
38968,Statistical Denoising for single molecule fluorescence microscopic images
38969,Blinking Molecule Tracking
38970,Towards a Mathematical Foundation of Immunology and Amino Acid Chains


Sentence transformers

# Sentence Transformers

In [ ]:
!pip install -U -q sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 52.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 36.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [ ]:
from sentence_transformers import SentenceTransformer, util

In [ ]:
model=SentenceTransformer('all-MiniLM-L6-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
sentences=df['titles']

In [ ]:
sentences

0                                                                       Survey on Semantic Stereo Matching / Semantic Depth Estimation
1        FUTURE-AI: Guiding Principles and Consensus Recommendations for Trustworthy Artificial Intelligence in Future Medical Imaging
2                                          Enforcing Mutual Consistency of Hard Regions for Semi-supervised Medical Image Segmentation
3                                                        Parameter Decoupling Strategy for Semi-supervised 3D Left Atrium Segmentation
4                                                       Background-Foreground Segmentation for Interior Sensing in Automotive Industry
                                                                     ...                                                              
38967                              A Ray-based Approach for Boundary Estimation of Fiber Bundles Derived from Diffusion Tensor Imaging
38968                                                  

In [ ]:
embeddings= model.encode(sentences)

In [ ]:
embeddings

array([[-3.73324528e-02, -5.35819796e-04, -4.86986861e-02, ...,
         4.00167964e-02, -8.07431061e-03, -1.12721235e-01],
       [-6.75695762e-02, -3.25633623e-02,  2.85481997e-02, ...,
         5.83391897e-02, -2.35455632e-02, -3.87291610e-02],
       [ 3.01970299e-02, -1.08661770e-04,  6.92958608e-02, ...,
         7.74508789e-02, -9.04723480e-02, -3.32056843e-02],
       ...,
       [-5.21421395e-02, -8.82680342e-02, -5.11411242e-02, ...,
        -6.05786778e-02,  5.92943132e-02, -2.94687543e-02],
       [-1.54421693e-02, -7.96080083e-02,  1.47850923e-02, ...,
         1.22725880e-02,  2.37942319e-02, -4.95932102e-02],
       [ 2.12828815e-02, -7.37964883e-02,  6.02626801e-02, ...,
         1.05841152e-01, -3.17206681e-02, -1.08712167e-01]], dtype=float32)

In [ ]:
embeddings.shape

(38972, 384)

In [ ]:
c = 0
#This loop iterates over pairs of sentences and their corresponding embeddings.
#zip is used to iterate over both lists simultaneously.
for sentence, embedding in zip(sentences, embeddings):
    print("Sentence:", sentence)
    print("Embedding length:", len(embedding)) # list of floats
    print("")
    # Breaks out of the loop after printing information for the first 5 sentences.
    if c >=5:
        break
    c +=1

Sentence: Survey on Semantic Stereo Matching / Semantic Depth Estimation
Embedding length: 384

Sentence: FUTURE-AI: Guiding Principles and Consensus Recommendations for Trustworthy Artificial Intelligence in Future Medical Imaging
Embedding length: 384

Sentence: Enforcing Mutual Consistency of Hard Regions for Semi-supervised Medical Image Segmentation
Embedding length: 384

Sentence: Parameter Decoupling Strategy for Semi-supervised 3D Left Atrium Segmentation
Embedding length: 384

Sentence: Background-Foreground Segmentation for Interior Sensing in Automotive Industry
Embedding length: 384

Sentence: EdgeFlow: Achieving Practical Interactive Segmentation with Edge-Guided Flow
Embedding length: 384



# Save Files

In [ ]:
import pickle

with open("/content/drive/MyDrive/ models/embeddings.pkl",'wb') as f:
  pickle.dump(embeddings,f)

with open("/content/drive/MyDrive/ models/sentences.pkl",'wb') as f:
  pickle.dump(sentences,f)

with open("/content/drive/MyDrive/ models/rec_model.pkl",'wb') as f:
  pickle.dump(model,f)

# Recommendation for similar papers

In [ ]:
import pickle
embeddings=pickle.load(open("/content/drive/MyDrive/ models/embeddings.pkl",'rb'))
sentences=pickle.load(open("/content/drive/MyDrive/ models/sentences.pkl",'rb'))
rec_model=pickle.load(open("/content/drive/MyDrive/ models/rec_model.pkl",'rb'))

In [ ]:
import torch

def recommendation(input_paper):
    # Calculate cosine similarity scores between the embeddings of input_paper and all papers in the dataset.
    cosine_scores = util.cos_sim(embeddings, rec_model.encode(input_paper))

    # Get the indices of the top-k most similar papers based on cosine similarity.
    top_similar_papers = torch.topk(cosine_scores, dim=0, k=5, sorted=True)

    # Retrieve the titles of the top similar papers.
    papers_list = []
    for i in top_similar_papers.indices:
        papers_list.append(sentences[i.item()])

    return papers_list

In [ ]:
#exampel usage 1: (use this paper as input (Attention is All you Need))
input_paper = input("Enter the title of any paper you like")
recommend_papers = recommendation(input_paper)

print("We recommend to read this paper............")
print("=============================================")
for paper in recommend_papers:
  print(paper)

Enter the title of any paper you likeSurvey on Semantic Stereo Matching / Semantic 
We recommend to read this paper............
Survey on Semantic Stereo Matching / Semantic Depth Estimation
Single View Stereo Matching
Stereo Matching Based on Visual Sensitive Information
Stereo Object Matching Network
Multi view stereo with semantic priors
